In [ ]:
!pip install onnx onnx-tf onnxruntime
!pip install onnx2tf tensorflowjs onnx-graphsurgeon sng4onnx

In [ ]:
!pip install onnx
!pip install onnxruntime
!pip install onnxruntime-gpu

## YOLO

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8l.pt")

# results = model("bus.jpg")

# for result in results:
#     result.save(filename="result.jpg")

In [ ]:
model.export(format="tflite")

In [ ]:
onnx_model = YOLO("yolov8l.onnx")

results = onnx_model("bus.jpg")

## Scikit-Learn

In [ ]:
!pip install skl2onnx

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# model = RandomForestClassifier()
# model = KNeighborsClassifier(n_neighbors=3)
model = MLPClassifier(random_state=1, max_iter=300)
model.fit(X_train, y_train)

In [ ]:
import skl2onnx

onnx_model = skl2onnx.to_onnx(model, X[0])
with open("scikit_kearn_onnx_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
import onnxruntime as ort

onnx_session = ort.InferenceSession(
    "scikit_kearn_onnx_model.onnx",
    providers=["CPUExecutionProvider"])

In [ ]:
input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name
input_name, output_name

In [ ]:
pred = onnx_session.run(
    ['output_label'],
    {'X': X_test[0]}
    )
pred

## PyTorch

In [ ]:
!pip install onnx
!pip install onnxscript

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

In [ ]:
onnx_program.save("my_image_classifier.onnx")

In [ ]:
import onnx
onnx_model = onnx.load("my_image_classifier.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
import onnxruntime as ort

onnx_session = ort.InferenceSession(
    "my_image_classifier.onnx",
    providers=["CPUExecutionProvider"])

input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name
# print(input_name, output_name)

onnx_input = torch_input.detach().cpu().numpy()

pred = onnx_session.run(
    [output_name],
    {input_name: onnx_input}
)

pred

In [ ]:
onnxruntime_outputs = pred

In [ ]:
torch_outputs = torch_model(torch_input)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

## TensorFlow

In [ ]:
!pip install tf2onnx

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50


model = ResNet50(weights="imagenet", include_top=True)

In [ ]:
import tf2onnx

input_data = tf.TensorSpec(
        model.inputs[0].shape,
        dtype=model.inputs[0].dtype,
        name=model.inputs[0].name
    )

onnx_model, _ = tf2onnx.convert.from_keras(model, [input_data])

In [ ]:
import onnx
onnx.save(onnx_model, "resnet50.onnx")

In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
from tensorflow.keras.applications.resnet50 import decode_predictions


onnx_session = ort.InferenceSession(
    "resnet50.onnx",
    providers=["CPUExecutionProvider"]
    )

input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name
# print(input_name, output_name)

# onnx_input = np.random.normal(size=[1, 224, 224, 3]).astype(np.float32)
image = Image.open("/content/2010-kodiak-bear-1.jpg")
image = image.resize((224, 224))  # Resize the image to 224x224 as expected by ResNet50
image = np.array(image).astype(np.float32)
image = np.expand_dims(image, axis=0)
onnx_input = image

pred = onnx_session.run(
    [output_name],
    {input_name: onnx_input}
)

In [ ]:
decode_predictions(pred[0])